In [1]:
import torch
from torch_geometric.data import Data
import pandas as pd
import numpy as np
edge_index= np.load("ppi_edge_list_mapped.npy")
embeddings_features=np.load("ppi_embeddings_features.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'ppi_edge_list_mapped.npy'

x: embedding with shape [num_nodes, num_node_features]
edge_index: graph connectivity in COO format with shape [2, num_edges]

Note that to account for undirected graph, each edge appears in reversed order twice in the transposed edge_index (each entry is a pair of edges)

In [ ]:
t = torch.from_numpy(edge_index)

In [ ]:
data = Data(x = torch.from_numpy(embeddings_features),edge_index=t.t().contiguous())

In [ ]:
# Model 
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 32)
        self.conv2 = GCNConv(32, 16)   #hm, could make # of color as the output dim here?

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        #x = x.to(dtype=torch.float32)
        # print((x.dtype))
        # print((edge_index.dtype))
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)
    def similarity(self, x, y, test=False):
      x, y = F.normalize(x), F.normalize(y)
      sim = x.mm(y.t())
      if test:
        # the testing code wants the index of the best match
        _, sim = sim.max(0)
      return sim

In [ ]:
from torch_geometric.loader import DataLoader,RandomNodeSampler, NeighborLoader
from torch_geometric.utils import structured_negative_sampling
loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=True, batch_size=10)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
model.train()
margin = 0.01
train_running_loss = 0.0
for sample in loader:
    # print(device)
    sample.x = sample.x.to(device).to(dtype=torch.float32)
    sample.x.requires_grad=True
    # sample.x = requires_grad=True
    sample.edge_index = sample.edge_index.to(device).long()
    # print(sample.x.is_cuda)
    # print(type(sample.edge_index))
    # print(sample.edge_index.is_cuda)
    optimizer.zero_grad()
    i, j, k = structured_negative_sampling(sample.edge_index)
    negatives = (i,k)   #not neighbors
    positives = (i,j)   #neighbors 
    output = model(sample)
    #pos = model.similarity(sample.x[i], sample.x[j])
    #neg = model.similarity(sample.x[i], sample.x[k])
    pos = model.similarity(output[i], output[j])
    neg = model.similarity(output[i], output[k])
    diff =pos.diag() -neg.diag() +margin      # Note for coloring, we want negatives closer and positives further
    triplet_loss_matrix = diff.mean()
    loss = triplet_loss_matrix
    loss.backward()
    optimizer.step()
    train_running_loss += loss.detach().item()